# Very simple startup with Spark (WORKING!!!)

In [4]:
import ujson
import ast

def json_parser(line):
    review_dictionary = ast.literal_eval(line)
    if 'price' in review_dictionary.keys():
        
        #yield "reviewerID", review_dictionary['reviewerID']
        #yield "asin", review_dictionary['asin']
        #yield "reviewerName", review_dictionary['reviewerName']
        #yield "helpful", review_dictionary['helpful']
        #yield "reviewText", review_dictionary['reviewText']
        yield "price", review_dictionary['price']
        #yield "summary", review_dictionary['summary']
        #yield "unixReviewTime", review_dictionary['unixReviewTime']
        #yield "reviewTime", review_dictionary['reviewTime']
        yield 'count', 1
    
lines = sc.textFile("../../amazon_dataset/metadata.json")
review_values = lines.flatMap(json_parser)
key_lists = review_values.reduceByKey(lambda a,b: a+b)

a = key_lists.collect()

print a

print a[0][1]*1.0/a[1][1]

[('count', 6063208), ('price', 211831713.8699847)]
0.0286227585531


In [57]:
result = {}
for elm in a:
    if elm[0]=='count':
        result['count']=elm[1]
    else:
        result['price']=elm[1]

In [58]:
print 'The average price of all the products is:',result['price']/result['count']

The average price of all the products is: 34.9372335354


In [1]:
music = spark.read.json('../../amazon_dataset/reviews_Musical_Instruments_5.json')
music.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [76]:
music.registerTempTable('music')
result = spark.sql("SELECT asin as ProductID,summary,helpful,unixReviewTime FROM music WHERE overall==5.0 AND helpful[1]-helpful[0]<10 AND helpful[0]!=0 AND helpful[1]!=0 AND helpful[0]>100")
result.show(10, truncate=False)

+----------+------------------------------------------------------------+----------+--------------+
|ProductID |summary                                                     |helpful   |unixReviewTime|
+----------+------------------------------------------------------------+----------+--------------+
|B0002GXZK4|This guitar DOES have a BIG SOUND for a small price         |[156, 160]|1111708800    |
|B0002H0H4A|The Pop Rocks with the Yeti                                 |[101, 105]|1342656000    |
|B000B6DHB2|Impressive Sound from an Inexpensive Source - UPDATE 7/11/14|[130, 136]|1230422400    |
|B000BWEM42|Well built folding stand great for guitar music and tab     |[156, 161]|1197504000    |
|B002Q0WSO8|Finally!  Amazon offers the Intelli IMT-500                 |[188, 193]|1177027200    |
|B003FVB7XA|Considerably better than anticipated!                       |[259, 266]|1223683200    |
|B0042EZH6W|Bought it for my Kid. Now I'm buying it for myself!         |[190, 192]|1293753600    |


In [43]:
filtered = music.filter(music.overall>3.5).show(10)

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0528881469| [0, 0]|    5.0|We got this GPS f...| 06 2, 2013| AO94DHGC771SJ|             amazdnu|     Gotta have GPS!|    1370131200|
|0594451647| [3, 3]|    5.0|I am using this w...| 01 3, 2014|A2JXAZZI9PHK9Z|Billy G. Noland "...|HDMI Nook adapter...|    1388707200|
|0594451647| [0, 0]|    5.0|This adaptor is r...| 05 4, 2014| AAZ084UMH8VZ2|D. L. Brown "A Kn...|A Perfdect Nook H...|    1399161600|
|0594451647| [0, 0]|    4.0|This adapter easi...|07 11, 2014| AEZ3CR6BKIROJ|        Mark Dietter|A nice easy to us...|    1405036800|
|0594451647| [3, 3]|    5.0|This product real...|01 20, 2014|A

In [46]:
music.drop('reviewerName')

DataFrame[asin: string, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, summary: string, unixReviewTime: bigint]

In [74]:
music.unixReviewTime.alias('Unix Review Time')

Column<unixReviewTime AS `Unix Review Time`>

In [87]:
music.select('asin').groupby('asin').count().sort('count',ascending=False).show()

+----------+-----+
|      asin|count|
+----------+-----+
|B003VWJ2K8|  163|
|B0002E1G5C|  143|
|B0002F7K7Y|  116|
|B003VWKPHC|  114|
|B0002H0A3S|   93|
|B0002CZVXM|   74|
|B0006NDF8A|   71|
|B0009G1E0K|   69|
|B0002E2KPC|   68|
|B0002GLDQM|   67|
|B004XNK7AI|   65|
|B005FKF1PY|   63|
|B00646MZHK|   62|
|B0002GMGYA|   58|
|B001PGXHX0|   58|
|B0018TIADQ|   57|
|B000978D58|   57|
|B001PGXKC8|   55|
|B0002D0CEO|   55|
|B000068NW5|   52|
+----------+-----+
only showing top 20 rows



# Creating a DataFrame from JSON string (useful for Metadata.json)

In [55]:
jsonStrings = ['{"name":"Yin","address":{"city":"Columbus","state":"Ohio"}}','{"name":"Yon","address":{"city":"Columbus","state":"Ohio"}}']
otherPeopleRDD = sc.parallelize(jsonStrings)
otherPeople = spark.read.json(otherPeopleRDD)
otherPeople.show()

+---------------+----+
|        address|name|
+---------------+----+
|[Columbus,Ohio]| Yin|
|[Columbus,Ohio]| Yon|
+---------------+----+



In [61]:
metadata = spark.read.json('../../amazon_dataset/metadata.json')
metadata.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Appliances: long (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Automotive: long

# IMPORTANT: Reviews Parsing and Querying

## Parsing

Using this function to properly parse each JSON dict:

In [2]:
import ujson
from toolz import dissoc
import datetime
import re
import time
from pyspark.sql.types import *

Function used to convert a UNIX timestamp to a proper datetime object (more human-readable):

In [3]:
def convert_date(date):
    date = datetime.date.fromtimestamp(int(date))
    return date

#Used to remove reviewerName (not interesting) and to convert UNIX timestamp in date object
def reviews_parser(line):
    try:
        line = ujson.loads(line)
        line['productID'] = line['asin']
        line['reviewTime'] = convert_date(line['unixReviewTime'])        
    except:
        line['reviewTime'] = None
    return dissoc(line,'reviewerName','unixReviewTime','asin')

We're using the `k_core_no_books.json` so we get rid of books, which are not relevant for our analysis. Amazon became something like a huge book shop and more than 50% of the total reviews are for books, so they're actually involving other parameters regarding the content of the book. We are focusing on reviews for each specific product.

In [4]:
f = '../../amazon_dataset/kcore_5_no_books.json'

#Definition of schema for the DataFrame
string_fields = ['productID','reviewText','reviewerID','summary']
fields = [StructField(field_name, StringType(), False) for field_name in string_fields]
fields+=[StructField('reviewTime',DateType(),True),
         StructField('overall',FloatType(),False),
         StructField('helpful',ArrayType(IntegerType(),containsNull=False),False)]
schema = StructType(fields)

data = sc.textFile(f)
data = data.map(reviews_parser)
data = data.toDF(schema)

In [5]:
data.printSchema()

root
 |-- productID: string (nullable = false)
 |-- reviewText: string (nullable = false)
 |-- reviewerID: string (nullable = false)
 |-- summary: string (nullable = false)
 |-- reviewTime: date (nullable = true)
 |-- overall: float (nullable = false)
 |-- helpful: array (nullable = false)
 |    |-- element: integer (containsNull = false)



In [5]:
data.show(10)

+----------+--------------------+--------------+--------------------+----------+-------+--------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall| helpful|
+----------+--------------------+--------------+--------------------+----------+-------+--------+
|1384719342|Not much to write...|A2IBPI20UZIR0U|                good|2014-02-28|    5.0|  [0, 0]|
|1384719342|The product does ...|A14VAT5EAX3D9S|                Jake|2013-03-16|    5.0|[13, 14]|
|1384719342|The primary job o...|A195EZSQDW3E21|It Does The Job Well|2013-08-28|    5.0|  [1, 1]|
|1384719342|Nice windscreen p...|A2C00NNG1ZQQG2|GOOD WINDSCREEN F...|2014-02-14|    5.0|  [0, 0]|
|1384719342|This pop filter i...| A94QU4C90B1AX|No more pops when...|2014-02-21|    5.0|  [0, 0]|
|B00004Y2UT|So good that I bo...|A2A039TZMZHH9Y|      The Best Cable|2012-12-21|    5.0|  [0, 0]|
|B00004Y2UT|I have used monst...|A1UPZM995ZAH90|Monster Standard ...|2014-01-19|    5.0|  [0, 0]|
|B00004Y2UT|I now us

This command is needed in order to register a table name like `reviews` which can be used directly in SQL queries.

In [6]:
data.registerTempTable('reviews')

Let's count the total number of reviews:

In [13]:
spark.sql('SELECT count(*) AS Total_Reviews FROM reviews').show()

+-------------+
|Total_Reviews|
+-------------+
|      9288692|
+-------------+



So, our dataset contains **9M+ reviews**.

## Trying to query along all the columns, to check if they work with their type:

**Date**: there are some problems, in the schema it's listed as 'date' type, instead it's impossible to extract year/month/day from the object (it raises exception); it works if it's treated as string object... WTF?

In [11]:
data.filter(data.reviewTime.startswith('2013-12-25')).show(10)

+----------+--------------------+--------------+--------------------+----------+-------+-------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall|helpful|
+----------+--------------------+--------------+--------------------+----------+-------+-------+
|B000068O1N|Haven't used it y...|A3UPEG3LRWOX16|         Nice Cables|2013-12-25|    4.0| [0, 0]|
|B0002E1G5C|Overall, I like t...|A19Q4B515ENF9C|Does the job but ...|2013-12-25|    4.0| [0, 0]|
|B0002E1I2I|I bought my new g...|A19Q4B515ENF9C|I guess this work...|2013-12-25|    3.0| [0, 0]|
|B0006NDF8A|This is fine for ...|A19Q4B515ENF9C|Works fine and pr...|2013-12-25|    5.0| [0, 0]|
|B002NG7DEK|This strap works ...|A19Q4B515ENF9C|Works great for m...|2013-12-25|    4.0| [0, 0]|
|B005LYIW3W|This pedal is ver...|A2W03JTG0K5OKF|         Great Tone!|2013-12-25|    5.0| [0, 0]|
|B0006IX7Y2|Easy to use, easy...|A2KS03QLEXP09W|       Great product|2013-12-25|    5.0| [0, 0]|
|B0007LVJ20|Easy to use. Keep.

Not working treating the reviewTime as a date object and trying to extract its attributes:

In [26]:
data.filter(data.reviewTime.year==2014).show(10)

AnalysisException: u"Can't extract value from reviewTime#78;"

Querying along **helpful**:

In [7]:
data.filter(data.helpful[1]-data.helpful[0]>20).show()

+----------+--------------------+--------------+--------------------+----------+-------+----------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall|   helpful|
+----------+--------------------+--------------+--------------------+----------+-------+----------+
|B0002CZV82|This is a cheap p...|A2B58VXLLOFQKR|         It distorts|2009-11-13|    1.0|   [2, 30]|
|B000T9NKEU|It was Not what I...|A3MJARDJ31M698|       Cost Too Much|2010-06-05|    1.0|   [0, 30]|
|B000ULAP4U|garbage if you li...|A1B3CNORXB1USI|almost zero stars...|2012-06-12|    1.0| [11, 101]|
|B0042F1L4S|My review(s) for ...|A2OPSPL8LSSJPC|this amp is ooooo...|2012-01-24|    1.0|   [5, 30]|
|B004OU2IQG|allotoday went to...|A1B3CNORXB1USI|so so for  conver...|2012-10-25|    1.0|  [17, 65]|
|B004TE5HBU|Just use a smartp...|A16Z3HTUIYPDH8|    Use a smartphone|2013-03-13|    1.0|   [5, 38]|
|B0002U2V1Y|I have a 1998 Ram...| A4WCZVA328QB4|Claying is more f...|2009-09-24|    3.0|[137, 177]|


Querying along **productID**:

In [ ]:
data.groupBy('productID').count().sort('count',ascending=False).show()

Querying along **reviewerID**:

In [36]:
data.groupBy('reviewerID').count().withColumnRenamed("count", "total").sort('total',ascending=False).filter('total%2==0').show(10)

+--------------+-----+
|    reviewerID|total|
+--------------+-----+
| A9Q28YTLYREO7|  578|
| A8IFUOL8S9BZC|  256|
|A1GN8UJIZLCA59|  246|
|A200C7YQJ45LRR|  200|
|A1J5KCZC8CMW9I|  182|
| A8SCX6VUTE05H|  172|
|A2582KMXLK2P06|  160|
|A1IKOYZVFHO1XP|  144|
|A12R54MKO17TW0|  132|
|A2ETZ7GF5B1712|  130|
+--------------+-----+
only showing top 10 rows



Querying along **reviewText** (same as **summary**):

In [12]:
data.filter(data['reviewText'].startswith('Shit')).sort('overall',ascending=False).show()

+----------+--------------------+--------------+--------------------+----------+-------+-------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall|helpful|
+----------+--------------------+--------------+--------------------+----------+-------+-------+
|B00GN8K6CU|Shit just got rea...|A1BQO66R6OLCCW|                 Wow|2013-11-19|    5.0| [1, 1]|
|B00HGOMTP2|Shit is off the c...|A1VT0G2BIAWMQJ|OMG SHELLI U DID DAT|2013-12-22|    5.0| [0, 0]|
|B00G9IYA5I|Shit just got rea...|A1BQO66R6OLCCW|               Omg!!|2013-10-31|    5.0| [3, 3]|
|B00FOQ6LW6|Shit just got rea...|A1BQO66R6OLCCW|                 Wow|2013-11-24|    5.0| [0, 0]|
|B00HFSMF76|Shit just got rea...|A1BQO66R6OLCCW|                 Wow|2014-01-16|    5.0| [0, 0]|
|B00F0UCDL4|Shit is about to ...|A1BQO66R6OLCCW|               Wow!!|2013-09-09|    5.0| [0, 0]|
|B0087ZUNR0|Shit just got rea...|A1BQO66R6OLCCW|                 Wow|2013-12-19|    5.0| [0, 0]|
|0756408148|Shit just got rea.

# Start of the interesting analysis

1) Reviews' helpfulness (most helpful?)

2) Most voted reviews (independent from good/bad score, just highest value for helpful[1])

3) Most appreciated products (highest overall average)[per category?]

4) Word clouds, both bad and good, both summary and text

5) Overall score and sentiment: do this two parameters agree with each other? *(Is the text of the review expressing the same sentiment of the text?)*

Temporal analysis:
* Months of the year with the most reviews, across years
* Sentiment analysis of the reviews over time (whole period, single year?)

## Reviews' Helpfulness
Which are the most helpful reviews ever? In order to say this we can take in consideration the `helpful` parameter: it is represented as an array of 2 elements, where the second one indicates how many people *voted* the review, and the first one tells how voters out of the total considered this review *helpful*. The difference between these two values, thus, tells how many considered this review not helpful.

Since there are lots of reviews with very few voters, we should filter them and take in consideration just reviews that were voted a lot of times. Let's calculate the **average number of voters** (we obviously exclude those reviews that were never voted):

In [9]:
spark.sql("SELECT avg(helpful[1]) FROM reviews WHERE helpful[0]>0 AND helpful[1]>0").show()

+-----------------+
|  avg(helpful[1])|
+-----------------+
|8.431077453916476|
+-----------------+



The result above means that there are much more reviews with a low number of voters, which is obviously acceptable. Instead, let's try to find the **top 20 reviews with the highest number of voters**:

In [15]:
spark.sql("SELECT * FROM reviews ORDER BY helpful[1] DESC").show()

+----------+--------------------+--------------+--------------------+----------+-------+--------------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall|       helpful|
+----------+--------------------+--------------+--------------------+----------+-------+--------------+
|B0047E0EII|For decades I hav...|A1TTA1UUGY4WY4|No more winning f...|2011-03-03|    5.0|[52176, 52861]|
|B0074BW614|I've been an iPad...|A3QCF8CVINEXB8|You Get What You ...|2012-09-17|    4.0|[30735, 31453]|
|B007FTE2VW|Fundamentally, Si...|A1DQOJ8PLXVPCO|    What a lousy toy|2013-03-06|    1.0|[10279, 10533]|
|B0029OSN4U|The primary diffe...|A3FKPBNI7UWQFW|Salon Laser Hair ...|2010-04-05|    4.0|  [9634, 9717]|
|B000FKBCX4|See those older 5...|A3284KYDZ00BZA|Dumbed down exper...|2008-09-07|    1.0|  [8606, 9403]|
|B00B2V66VS|This game is the ...|A1SIOM22E22TYY|THIS GAME IS AWES...|2013-01-24|    5.0|  [8116, 9258]|
|B008PPGFZG|If you DO NOT hav...| AK7T18HR8HYU2|READ APP DESCRIP

We come up to an already interesting result: 

***Most of the top 20 most voted reviews were positive: their overall score is above 4.0; But there are two outliers, the 3rd and 5th reviews in the list: these reviews have an overall score of 1.0! ***

* It's evident that all these reviews are pretty much **recent**, most of them were written in the time range from 2010 to 2013, with a few reviews in 2007, 2008 and 2009.
* People tend to vote **much more good reviews rather than bad reviews**.
* However, there are few examples of **bad reviews** who got **A LOT of voters**, mostly supporting their opinion.
* Since Amazon is showing first the top reviews for each product, it comes quite natural that once a review becomes a "top review", its voters grow exponentially. That's why the number of voters of the first most voted reviews differ by such a high gap.
* Why did those top-voted reviews actually become *top*? We can take in consideration the review text and analyze it... 

Do the voters agree with the reviewer? Actually this last query just showed which are the *most voted reviews*, but didn't explicitly show *how many voters actually considered that review helpful*.
Thus, we're going to show now the **average of how many voters considered the review NOT helpful**, and then compare it to the **average of how many voters DO considered the review helpful**:

In [10]:
spark.sql("SELECT avg(helpful[1]-helpful[0]) AS Avg_Not_Helpful_Voters FROM reviews WHERE helpful[0]>0 AND helpful[1]>0").show()

+----------------------+
|Avg_Not_Helpful_Voters|
+----------------------+
|     1.833070324557537|
+----------------------+



In [11]:
spark.sql("SELECT avg(helpful[0]) AS Avg_Helpful_Voters FROM reviews WHERE helpful[0]>0 AND helpful[1]>0").show()

+------------------+
|Avg_Helpful_Voters|
+------------------+
|6.5980071293589395|
+------------------+



It's clear that **most of the voters do agree with the review**.

Let's look instead at those reviews with the **highest number of voters who STRONGLY didn't agree with the reviewer** (*so didn't consider the review helpful*):

In [12]:
spark.sql("SELECT * FROM reviews ORDER BY helpful[1]-helpful[0] DESC").show()

+----------+--------------------+--------------+--------------------+----------+-------+------------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall|     helpful|
+----------+--------------------+--------------+--------------------+----------+-------+------------+
|B00AKIPBNS|They didnt make e...|A169RTOHC2SJPI|OUT OF STOCK!!!?????|2013-02-04|    1.0|  [20, 1853]|
|B00H5RYIBI|Often times, I lo...|A3CPJYAQMCUGNR|Purchased as a gi...|2014-01-02|    1.0|  [21, 1589]|
|B00IWULQQ2|Best look to the ...|A17XJCP9P9VSL0|Fictional fluff w...|2014-05-22|    1.0|  [24, 1250]|
|B00B2V66VS|This game is the ...|A1SIOM22E22TYY|THIS GAME IS AWES...|2013-01-24|    5.0|[8116, 9258]|
|B001KVZ6HK|Storytelling as a...|A3DGVB3T5QJNRE|About as Exciting...|2012-05-11|    1.0|  [33, 1170]|
|B005J6U77Q|I need to augment...|A3GGD7ZWVWIKOU|Flaccid Logos Spe...|2011-09-08|    1.0|  [24, 1057]|
|B0001VL0K2|Come on now New L...|A22VNXHU6IZ5MT|Come on New Line,...|2009-04-16|  

Actually we can do an interesting consideration: the output above clearly shows a **majority of negative reviews** (overall 1.0) than positive reviews (overall 5.0):
* this leads to state that **there are lots of negative reviews that a lot of people disagree with**, and...
* **there are some positive reviews with a relative high number of people who disagree with it**

If we carefully look at those **positive reviews**, it immediately points out that **the number of people who disagree is often much smaller than the number of people who agree**, and mostly we're talking about thousands of voters. Thus, we can simply state that this difference is acceptable and they can still be considered top reviews.

By the other hand, the **negative reviews** are quite different than the previous ones: we can divide them in two subsets:
* negative reviews with high number of voters and very small number of people who considered them helpful: possible candidates for no-sense reviews, they don't characterize very well the related product;
* negative reviews with high number of voters and people who considered them helpful: as well as we stated for the positive reviews, these can be considered top reviews, so they strongly characterize the product, even if in a bad way.

Therefore, in order to see **which top-reviews** (in terms of *number of voters*) **also gained the highest number of opinion supporters** (*people who agree with the review and the vote and think that the review is helpful*), we have to check where the **difference between** `helpful[1]` and `helpful[0]` **is minimum**. But, differently than the last query, we cannot simply revert the `ORDER BY` clause, because we might find many reviews with a very small number of voters.

We can assume it's reasonable to *filter* the reviews and take in consideration only those ones with *more than 3000 voters* (probably this will cut off some valid results...):

In [16]:
data.filter(data.helpful[1]>3000).sort(data.helpful[1]-data.helpful[0]).show()

+----------+--------------------+--------------+--------------------+----------+-------+------------+
| productID|          reviewText|    reviewerID|             summary|reviewTime|overall|     helpful|
+----------+--------------------+--------------+--------------------+----------+-------+------------+
|B0015EWMX8|I actually could ...|A27C6W7DLDD7QK|    Garmin Nuvi 255w|2008-06-30|    5.0|[3052, 3067]|
|B0002IQJ8W|This was my fourt...|A3RMAWK4ELOLRZ|          Mean Girls|2006-01-10|    5.0|[3013, 3038]|
|B002HWJT1A|Bose QC15 vs. Sen...|A2QDYGI5OETYV3|Bose QC15 vs. Sen...|2009-12-11|    5.0|[3198, 3225]|
|B0000U1OCI|Since getting att...|  A6FIAB28IS79|Best of Breed: Th...|2004-09-04|    5.0|[3443, 3475]|
|B004GFN2ZA|I thought it migh...|A1ITCVZ46MUB3A|Review from a Bio...|2011-11-27|    4.0|[3410, 3442]|
|B0023B14TK|If I'm going to s...|A17V9XL4CWTQ6G|The camcorder SHO...|2009-11-07|    5.0|[3553, 3585]|
|B000UUBCNO|My wife and I dec...| AC1YLEFC9AN5X|Great but not per...|2007-09-16|  

Now we can **analyze the text** of the reviews that appeared in the previous results and provide some specific examples for each of the subsets that we identified: